<a href="https://colab.research.google.com/github/swethasrihari/NLP/blob/main/NLP_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
training_path = "/content/drive/MyDrive/Colab Notebooks/sentiment_train.json"
test_path = "/content/drive/MyDrive/Colab Notebooks/sentiment_test.json"

train_data = pd.read_json(training_path, lines=True)
test_data = pd.read_json(test_path,lines=True)

Extracting review_titles and stars column data from the dataset

In [ ]:
def extract_required_columns(dataset):
  stars = []
  review_title = []
  for record in dataset.values:
    stars.append(record[3])
    review_title.append(record[5])
  return stars,review_title

In [ ]:
stars, review_title = extract_required_columns(train_data)
test_stars, test_review_title = extract_required_columns(test_data)

In [ ]:
import string

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text

Defining a function to split the given review into uni/bi/trigrams..

In [ ]:
import nltk
from nltk.util import ngrams

def split_into_ngrams(words, n):
  grams = []
  for i in range(len(words)):
    grams.append(list(ngrams(words[i],n)))
  return grams


Preprocessing data by removing punctuations, converting into lower case and splitting into words

In [ ]:
def preprocessing_data(review_title):
  words = []
  for review in review_title:
    cleaned_review = (remove_punctuation(review)).lower()
    words.append(cleaned_review.split())
  return words

In [ ]:
cleaned_train_review_title = preprocessing_data(review_title)
cleaned_test_review_title = preprocessing_data(test_review_title)
data = {"review_title":cleaned_train_review_title,
        "stars": stars}
df = pd.DataFrame(data)
testing_data = {"review_title":cleaned_test_review_title,
                "stars":test_stars}
test_df = pd.DataFrame(testing_data)
#print(test_df)

In [ ]:
def get_ngrams(words):
  bigrams = split_into_ngrams(words,2)
  trigrams = split_into_ngrams(words,3)
  return bigrams,trigrams



In [ ]:
train_bigrams,train_trigrams = get_ngrams(cleaned_train_review_title)
test_bigrams,test_trigrams = get_ngrams(cleaned_test_review_title)


In [ ]:
df['bigrams'] = train_bigrams
df['trigrams'] = train_trigrams
test_df['bigrams'] = test_bigrams
test_df['trigrams'] = test_trigrams
#print(df)

Converting the cleaned review title into vector for both train and test data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
flattened_train_review_title = [' '.join(review) for review in cleaned_train_review_title]
flattened_test_review_title = [' '.join(review) for review in cleaned_test_review_title]
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(flattened_train_review_title)
x_test = vectorizer.transform(flattened_test_review_title)

Fitting the data into Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
y_train = stars
y_test = test_stars
reg = LogisticRegression().fit(x_train, y_train)
y_pred = reg.predict(x_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print("Logistic Regression Classifier Metrics")
print(report)

Logistic Regression Classifier Metrics
              precision    recall  f1-score   support

           1       0.93      0.92      0.93      1000
           5       0.92      0.93      0.93      1000

    accuracy                           0.93      2000
   macro avg       0.93      0.93      0.93      2000
weighted avg       0.93      0.93      0.93      2000



In [ ]:
from sklearn.metrics import confusion_matrix
print("Confusion Matrix For Logistic Regression Model")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix For Logistic Regression Model
[[924  76]
 [ 72 928]]


Randomizing and reducing the length of the dataset to 10000


In [ ]:
data = df[['stars','review_title']]

import random
reduced_train = data.sample(n=10000)


In [ ]:
flattened_train_review_title = [' '.join(review) for review in reduced_train['review_title']]
flattened_test_review_title = [' '.join(review) for review in test_df['review_title']]


In [ ]:
x_reduced_train = vectorizer.fit_transform(flattened_train_review_title)
x_test = vectorizer.transform(flattened_test_review_title)

y_reduced_train = reduced_train['stars']
y_test = test_stars
reg = LogisticRegression().fit(x_reduced_train, y_reduced_train)
y_pred = reg.predict(x_test)

report = classification_report(y_test, y_pred)
print("Logistic Regression Classifier Metrics for 10000 training data")
print(report)
print("Confusion Matrix For Logistic Regression Model")
print(confusion_matrix(y_test, y_pred))

Logistic Regression Classifier Metrics for 10000 training data
              precision    recall  f1-score   support

           1       0.91      0.90      0.91      1000
           5       0.90      0.92      0.91      1000

    accuracy                           0.91      2000
   macro avg       0.91      0.91      0.91      2000
weighted avg       0.91      0.91      0.91      2000

Confusion Matrix For Logistic Regression Model
[[901  99]
 [ 84 916]]


Reducing the dataset to 5000

In [ ]:
reduced_train = data.sample(n=5000)
flattened_train_review_title = [' '.join(review) for review in reduced_train['review_title']]
x_train = vectorizer.fit_transform(flattened_train_review_title)
x_test = vectorizer.transform(flattened_test_review_title)
y_reduced_train = reduced_train['stars']
print(x_train.shape)
print(x_test.shape)

(5000, 3318)
(2000, 3318)


In [ ]:
reg = LogisticRegression().fit(x_train, y_reduced_train)
y_pred = reg.predict(x_test)
report = classification_report(y_test, y_pred)
print("Logistic Regression Classifier Metrics for 5000 training data")
print(report)
print("Confusion Matrix For Logistic Regression Model")
print(confusion_matrix(y_test, y_pred))

Logistic Regression Classifier Metrics for 5000 training data
              precision    recall  f1-score   support

           1       0.88      0.90      0.89      1000
           5       0.90      0.88      0.89      1000

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000

Confusion Matrix For Logistic Regression Model
[[903  97]
 [123 877]]


Naive Bayes Model

In [ ]:
flattened_train_review_title = [' '.join(review) for review in cleaned_train_review_title]

x_train = vectorizer.fit_transform(flattened_train_review_title)
x_test = vectorizer.transform(flattened_test_review_title)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

           1       0.90      0.92      0.91      1000
           5       0.92      0.90      0.91      1000

    accuracy                           0.91      2000
   macro avg       0.91      0.91      0.91      2000
weighted avg       0.91      0.91      0.91      2000



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[923  77]
 [104 896]]


Randomizing and reducing the length of the dataset to 10000


In [ ]:
flattened_train_review_title = [' '.join(review) for review in reduced_train['review_title']]
x_train = vectorizer.fit_transform(flattened_train_review_title)
x_test = vectorizer.transform(flattened_test_review_title)
print(x_train.shape)
print(x_test.shape)

(10000, 4952)
(2000, 4952)


In [ ]:
clf.fit(x_train, y_reduced_train)
y_pred = clf.predict(x_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           1       0.89      0.91      0.90      1000
           5       0.91      0.89      0.90      1000

    accuracy                           0.90      2000
   macro avg       0.90      0.90      0.90      2000
weighted avg       0.90      0.90      0.90      2000



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[918  82]
 [139 861]]


Reducing the dataset to 50000

In [ ]:
reduced_train = data.sample(n=50000)
flattened_train_review_title = [' '.join(review) for review in reduced_train['review_title']]
x_train = vectorizer.fit_transform(flattened_train_review_title)
x_test = vectorizer.transform(flattened_test_review_title)
y_reduced_train = reduced_train['stars']
print(x_train.shape)
print(x_test.shape)

(50000, 11427)
(2000, 11427)


In [ ]:
clf.fit(x_train, y_reduced_train)
y_pred = clf.predict(x_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           1       0.90      0.91      0.91      1000
           5       0.91      0.90      0.91      1000

    accuracy                           0.91      2000
   macro avg       0.91      0.91      0.91      2000
weighted avg       0.91      0.91      0.91      2000



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[918  82]
 [139 861]]


reducing the dataset to 5000

In [ ]:
reduced_train = data.sample(n=5000)
flattened_train_review_title = [' '.join(review) for review in reduced_train['review_title']]
x_train = vectorizer.fit_transform(flattened_train_review_title)
x_test = vectorizer.transform(flattened_test_review_title)
y_reduced_train = reduced_train['stars']
print(x_train.shape)
print(x_test.shape)

(5000, 3228)
(2000, 3228)


In [ ]:
clf.fit(x_train, y_reduced_train)
y_pred = clf.predict(x_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           1       0.87      0.92      0.89      1000
           5       0.91      0.86      0.89      1000

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[918  82]
 [139 861]]




* For Naive Bayes there is not much difference in the confusion matrix when we decrease the number of rows in a dataset.
*   Very slight difference in Precision, recall, F!-score and accuracy.


*   For Logistic regression, There was a huge difference in the confussion matrix, as we reduce the number of rows in the dataset, the values of true possitive and true negative decreases and the values of false possitive and false negative increases.
*   There was around 5% difference between the different metrices of datasets of 80000 and 5000 rows.





Resources Used:


ChatGPT


https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html


Stack Overflow

In [ ]:
#!sudo apt-get update
#!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-latex-recommended
#!pip install nbconvert
!jupyter nbconvert --to pdf "/content/drive/MyDrive/Colab Notebooks/NLP_HW1.ipynb"
# The semicolon below suppresses this cell's output
# This is to prevent 10 pages of installation messages from printing to the pdf
;

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/NLP_HW1.ipynb to pdf
[NbConvertApp] Writing 53929 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 51171 bytes to /content/drive/MyDrive/Colab Notebooks/NLP_HW1.pdf


''